In [9]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pdb
import math
import re

In [10]:
def dist(x,c):
    ndata,dimx = x.shape
    ncenters, dimc = c.shape

    xsum = np.sum(x**2,axis = 1)
    xsum = xsum[:,np.newaxis]
    csum = np.sum(c**2,axis = 1)
    csum = csum[:,np.newaxis]

    n2 =  xsum.dot(np.ones([1,ncenters]))+ np.ones([ndata,1]).dot(csum.T)- 2*x.dot(c.T)
    return n2

In [11]:
file = sio.loadmat('bodyfat_data.mat')
x = file["X"]
y = file["y"]
x_train = x[0:150, :]
x_test = x[150:, :]
y_train = y[0:150, :]
y_test = y[150:, :]
lam = 0.003
sigma = 15
n, d = x_train.shape

In [12]:
x_train.shape

(150, 2)

In [13]:
# KRR without offset
K_temp = dist(x_train, x_train)/(-2*sigma**2)
K=[]
for i in K_temp:
    K_sub = [math.exp(j) for j in i]
    K.append(K_sub)
K = np.array(K)
alpha = np.linalg.inv(K+np.eye(n)*lam*n)@y_train

K_test_temp = dist(x_train, x_test)/(-2*sigma**2)
K_test = []
for i in K_test_temp:
    K_test_sub = [math.exp(j) for j in i]
    K_test.append(K_test_sub)
K_test = np.array(K_test)

In [14]:
y_pre_train = alpha.T@K
y_pre_test = alpha.T@K_test
error_train = np.average((y_pre_train-y_train.T)**2, axis=1)
error_test = np.average((y_pre_test-y_test.T)**2, axis=1)
print("KRR without offset")
print("training error: ", error_train)
print("test error: ", error_test)

KRR without offset
training error:  [19.2495644]
test error:  [26.39690561]


In [15]:
# KRR with offset b
n_test, d = x_test.shape
O = np.ones((n,n))*(1/n)
K_centered = K - K@O - O@K + O@K@O

O_test = np.ones((n,n_test))*(1/n)
K_test_centered = K_test - O@K_test - K@O_test + O@K@O_test

y_train_bar = np.average(y_train, axis=0)
y_train_centered = y_train - y_train_bar
y_offset_pre_train = y_train_bar + y_train_centered.T\
                     @np.linalg.inv(K_centered + (n*lam)*np.eye(n))@K_centered
y_offset_pre_test = y_train_bar + y_train_centered.T\
                     @np.linalg.inv(K_centered + (n*lam)*np.eye(n))@K_test_centered

O_b = np.ones((n,1))*(1/n)

b = y_train_bar - y_train_centered.T@np.linalg.inv(K_centered+np.eye(n)*n*lam)@(K@O_b-O@K@O_b)

In [16]:

error_train = np.average((y_offset_pre_train-y_train.T)**2, axis=1)
error_test = np.average((y_offset_pre_test-y_test.T)**2, axis=1)
print("KRR with offset")
print("training error: ", error_train)
print("test error: ", error_test)
print("b:",b)

KRR with offset
training error:  [19.01686065]
test error:  [23.53586025]
b: [[19.42947537]]


It seems like the offset makes little contribution to the final performance in KRR model.